# Final clean and then reduce data down to <25mb. Removed all rows less than 4000 trips as not interested in these trips.

In [1]:
import pandas as pd
import os
import io
import numpy as np
from matplotlib import pyplot as plt
import importlib.resources

In [2]:
path = r"C:\Users\ryani\Desktop\JupyterLab\NY_Citibike_2022"

In [12]:
df = pd.read_pickle(os.path.join(path, '02_Prepared_Data', 'final_df.pkl'))
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,member_casual,date,Precipitation,avgtemp,trips_per_day,tripduration,tripduration(mins),usertype,month,value
0,BFD29218AB271154,electric_bike,2022-01-21 13:13:43.392,2022-01-21 13:22:31.463,West End Ave & W 107 St,7650.05,Mt Morris Park W & W 120 St,7685.14,40.802117,-73.968181,...,member,2022-01-21,0.0,-6.0,33598,528,9,member_electric_bike,1,1
1,7C953F2FD7BE1302,classic_bike,2022-01-10 11:30:54.162,2022-01-10 11:41:43.422,4 Ave & 3 St,4028.04,Boerum Pl\t& Pacific St,4488.09,40.673746,-73.985649,...,member,2022-01-10,0.0,1.6,34727,649,11,member_classic_bike,1,1
2,95893ABD40CED4B8,electric_bike,2022-01-26 10:52:43.096,2022-01-26 11:06:35.227,1 Ave & E 62 St,6753.08,5 Ave & E 29 St,6248.06,40.761227,-73.960940,...,member,2022-01-26,0.0,-2.3,42005,832,14,member_electric_bike,1,1
3,F853B50772137378,classic_bike,2022-01-03 08:35:48.247,2022-01-03 09:10:50.475,2 Ave & E 96 St,7338.02,5 Ave & E 29 St,6248.06,40.783964,-73.947167,...,member,2022-01-03,0.0,1.4,33189,2102,35,member_classic_bike,1,1
4,7590ADF834797B4B,classic_bike,2022-01-22 14:14:23.043,2022-01-22 14:34:57.474,6 Ave & W 34 St,6364.10,5 Ave & E 29 St,6248.06,40.749640,-73.988050,...,member,2022-01-22,0.0,-5.9,31969,1234,21,member_classic_bike,1,1


In [13]:
# Build a Series of counts for each trip pair
pair_counts = (df.value_counts(subset=['start_station_name', 'end_station_name']).rename('trip_count'))

# Map those counts back to df
df['trip_count'] = df.set_index(['start_station_name', 'end_station_name']) \
                     .index.map(pair_counts) \
                     .fillna(0) \
                     .astype(int)

In [14]:
df.shape

(29838166, 23)

In [15]:
df.drop_duplicates(inplace=True)

In [16]:
df.dropna(inplace=True)

In [17]:
df.shape

(29800943, 23)

In [18]:
df.describe()

,started_at,ended_at,start_lat,start_lng,end_lat,end_lng,date,Precipitation,avgtemp,trips_per_day,tripduration,tripduration(mins),month,value,trip_count
count,29800943,29800943,2.980094e+07,2.980094e+07,2.980094e+07,2.980094e+07,29800943,2.980094e+07,2.980094e+07,2.980094e+07,2.980094e+07,2.980094e+07,2.980094e+07,29800943.0,2.980094e+07
mean,2022-07-18 09:44:21.782483200,2022-07-18 10:00:39.443932416,4.074057e+01,-7.397526e+01,4.074035e+01,-7.397535e+01,2022-07-17 19:10:23.935514880,2.364417e+00,1.712427e+01,9.519982e+04,9.776615e+02,1.629346e+01,7.046107e+00,1.0,3.644096e+02
min,2022-01-01 00:00:13.532000,2022-01-01 00:02:15.504000,4.062737e+01,-7.402802e+01,4.056000e+01,-7.428000e+01,2022-01-01 00:00:00,0.000000e+00,-1.170000e+01,2.848000e+03,-3.437000e+03,-5.700000e+01,1.000000e+00,1.0,1.000000e+00
25%,2022-05-14 19:59:00.669999872,2022-05-14 20:16:48.159500032,4.071606e+01,-7.399379e+01,4.071602e+01,-7.399380e+01,2022-05-14 00:00:00,0.000000e+00,1.140000e+01,7.487300e+04,3.480000e+02,6.000000e+00,5.000000e+00,1.0,5.700000e+01
50%,2022-07-23 02:22:05.193999872,2022-07-23 02:44:54.612000,4.073936e+01,-7.398166e+01,4.073932e+01,-7.398166e+01,2022-07-23 00:00:00,0.000000e+00,1.790000e+01,1.036390e+05,6.090000e+02,1.000000e+01,7.000000e+00,1.0,1.690000e+02
75%,2022-09-26 13:58:57.725499904,2022-09-26 14:15:51.051500032,4.076319e+01,-7.395928e+01,4.076313e+01,-7.395943e+01,2022-09-26 00:00:00,8.000000e-01,2.410000e+01,1.173210e+05,1.077000e+03,1.800000e+01,9.000000e+00,1.0,4.090000e+02
max,2022-12-31 23:58:19.206000,2022-12-31 23:59:55.708000,4.088398e+01,-7.387859e+01,4.106000e+01,-7.375000e+01,2022-12-31 00:00:00,4.500000e+01,3.130000e+01,1.350720e+05,2.426886e+07,4.044810e+05,1.200000e+01,1.0,1.204100e+04
std,NaN,NaN,3.891113e-02,2.580035e-02,3.885548e-02,2.582301e-02,NaN,6.064377e+00,8.551268e+00,2.787221e+04,1.486671e+04,2.477788e+02,2.911559e+00,0.0,6.538854e+02


In [19]:
df.to_pickle(os.path.join(path, '02_Prepared_Data', 'final_df_clean.pkl'))

## Drop all rows with counts less than 4000 trips as am not intested in lesser used stations

In [25]:
df = pd.read_pickle(os.path.join(path, '02_Prepared_Data', 'final_df_clean.pkl'))

In [26]:
df.drop(df[df['trip_count'] < 4000].index, inplace=True)

In [31]:
df.shape

(167940, 23)

In [32]:
df.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'date', 'Precipitation', 'avgtemp', 'trips_per_day',
       'tripduration', 'tripduration(mins)', 'usertype', 'month', 'value',
       'trip_count'],
      dtype='object')

In [33]:
df = df[['start_station_name', 'end_station_name', 'start_lat', 'start_lng', 'end_lat', 'end_lng', 'date', 'avgtemp', 'trips_per_day', 'tripduration(mins)', 'usertype', 'month', 'trip_count']]

In [34]:
df.head()

,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,date,avgtemp,trips_per_day,tripduration(mins),usertype,month,trip_count
27079,Norfolk St & Broome St,Henry St & Grand St,40.717227,-73.988021,40.714211,-73.981095,2022-01-20,5.1,33928,3,member_classic_bike,1,4883
37650,Central Park North & Adam Clayton Powell Blvd,Central Park North & Adam Clayton Powell Blvd,40.799484,-73.955613,40.799484,-73.955613,2022-01-16,-8.0,20930,4,casual_classic_bike,1,4260
37652,Central Park North & Adam Clayton Powell Blvd,Central Park North & Adam Clayton Powell Blvd,40.799484,-73.955613,40.799484,-73.955613,2022-01-16,-8.0,20930,54,casual_classic_bike,1,4260
37653,Central Park North & Adam Clayton Powell Blvd,Central Park North & Adam Clayton Powell Blvd,40.799484,-73.955613,40.799484,-73.955613,2022-01-16,-8.0,20930,55,casual_classic_bike,1,4260
37654,Central Park North & Adam Clayton Powell Blvd,Central Park North & Adam Clayton Powell Blvd,40.799484,-73.955613,40.799484,-73.955613,2022-01-16,-8.0,20930,5,casual_electric_bike,1,4260


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 167940 entries, 27079 to 29830894
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   start_station_name  167940 non-null  object        
 1   end_station_name    167940 non-null  object        
 2   start_lat           167940 non-null  float64       
 3   start_lng           167940 non-null  float64       
 4   end_lat             167940 non-null  float64       
 5   end_lng             167940 non-null  float64       
 6   date                167940 non-null  datetime64[ns]
 7   avgtemp             167940 non-null  float64       
 8   trips_per_day       167940 non-null  int64         
 9   tripduration(mins)  167940 non-null  int64         
 10  usertype            167940 non-null  category      
 11  month               167940 non-null  int32         
 12  trip_count          167940 non-null  int32         
dtypes: category(1), datetime64[n

In [36]:
for col in df.select_dtypes(include=['int64']):
    df[col] = pd.to_numeric(df[col], downcast='unsigned')
for col in df.select_dtypes(include=['float64']):
    df[col] = pd.to_numeric(df[col], downcast='float')

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 167940 entries, 27079 to 29830894
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   start_station_name  167940 non-null  object        
 1   end_station_name    167940 non-null  object        
 2   start_lat           167940 non-null  float32       
 3   start_lng           167940 non-null  float32       
 4   end_lat             167940 non-null  float32       
 5   end_lng             167940 non-null  float32       
 6   date                167940 non-null  datetime64[ns]
 7   avgtemp             167940 non-null  float32       
 8   trips_per_day       167940 non-null  uint32        
 9   tripduration(mins)  167940 non-null  uint16        
 10  usertype            167940 non-null  category      
 11  month               167940 non-null  int32         
 12  trip_count          167940 non-null  int32         
dtypes: category(1), datetime64[n

In [39]:
df.to_csv(os.path.join(path, '02_Prepared_Data', 'sampled_top_trips.csv'))